***
## Передаём данные из view-функции в HTML-шаблон

Для передачи информации в шаблон используется третий (необязательный) параметр функции `render()`. 

Это происходит так:

1. **Подготовка информации для страницы**: все данные, которые должны быть опубликованы на определённой странице, подготавливаются во view-функции и сохраняются в словарь — обычно его называют `context`.

2. **Отправка информации в шаблон**: при вызове функции `render()` в неё можно передать не только объект запроса и адреса шаблона, но и словарь, информацию из которого нужно разместить в шаблоне: `render(request, template_name, context)`.

3. **Вывод информации в шаблон**: в шаблоне подготавливаются места, куда должны быть выведены данные из переданного словаря. При рендеринге страницы шаблонизатор выведет содержимое словаря `context` в указанные места шаблона.

```py
# homepage/views.py
...

def index(request):
    # Адрес шаблона сохранён в переменную, это не обязательно, но удобно.
    template_name = 'homepage/index.html'
    # Строка, которую надо вывести на страницу, тоже сохранена в переменную.
    title = 'Главная страница ACME'
    promo_product = 'Iron carrot'
    context = {
        # Ключ словаря и имя переменной не обязательно называть одинаково,
        # но обычно это удобнее, чем использовать два разных имени.
        'title': title,
        'promo_product': promo_product,
    }
    # Третьим аргументом в render() передаём словарь context:
    return render(request, template_name, context)
```

Чтобы вывести в HTML-шаблон данные из словаря `context`, нужно лишь указать ключи словаря `title` и `promo_product` в нужном месте шаблона.

Ключ указывают в двойных фигурных скобках: `{{ key }}`. По кодстайлу Django между именем ключа и фигурными скобками обязательно надо ставить пробелы.

```html
<!-- templates/homepage/index.html -->
...
<h1>{{ title }}</h1>
...
<p>Товар дня: {{ promo_product }}</p> 
...
```

![alt text](https://pictures.s3.yandex.net/resources/image_1744207880.png)

> В качестве значений словаря `context` можно передавать любые данные: строки, списки, словари, любые другие объекты.

Шаблонизатор Django упрощает и унифицирует обращение к свойствам и ключам переданных объектов. Он на лету определяет, что следует вызвать: свойство объекта, значение по ключу из словаря или метод. 

Обратиться к элементу словаря, атрибуту объекта или элементу списка можно через точечную нотацию:

***
```html
{{ var_dict.key }} — обращение к ключу вложенного словаря
{{ var_instance.attribute }} — обращение к атрибуту или методу класса
{{ var_list.0 }} — обращение к элементу списка 
```
***

## Ветвление

Шаблонизатор Django поддерживает и ветвление. Тег ветвления `{% if %}` очень похож на инструкцию `if...elif...else` в языке Python. 

Обратите внимание: у тега `{% if %}` обязательно должен быть закрывающий тег `{% endif %}`:

***
```html
{% if promo_product == "Iron carrot" %}
  Железная морковка — хорошо для стальных зубов.
{% elif promo_product == "Giant mousetrap" %}
  Ребята, давайте жить дружно!
{% else %}
  Вы все меня ещё не знаете, но вы меня узнаете!
{% endif %}
```
***

Тот или иной вариант текста будет выведен в зависимости от того, какое значение передано в словаре `context` под ключом `promo_product`.

В шаблонах работают и другие логические операторы, и операторы сравнения. [Документация Django рассказывает о них](https://docs.djangoproject.com/en/3.2/ref/templates/builtins/#if) достаточно подробно.

***
## Циклы

По итерируемым объектам можно пройтись циклом прямо в шаблоне. Синтаксис очень похож на аналогичные конструкции в Python. 

В словаре `context` под ключом `products` передаём в шаблон список товаров:

***
```py
# catalog/views.py
...
def product_list(request):
    template_name = 'catalog/product_list.html'
    title = 'Список товаров ACME'
    products = [
      'Iron carrot',
      'Giant mousetrap',  
      'Dehydrated boulders',
      'Invisible paint',
    ]
    context = {
        'title': title,
        'products': products,
    }
    return render(request, template_name, context)
```
***

Этот список можно перебрать циклом `{% for ... in ... %}` прямо в шаблоне:

***
```html
<!-- templates/catalog/product_list.html -->
...

<h1>{{ title }}</h1>  
{% for product in products %}
  <p>{{ product }}</p>
{% endfor %}
```
***

## Дополнительные инструменты в циклах {% for ... in ... %}

В циклах, применяемых в шаблонах Django, доступна переменная `forloop` — объект с несколькими полезными атрибутами:

`forloop.counter` — номер текущей итерации цикла (начинает отсчёт с 1).

`forloop.counter0` — номер текущей итерации цикла «по-программистски» (начинает отсчёт с 0).

`forloop.revcounter` — количество итераций, оставшихся до конца цикла (на последнем проходе цикла этот атрибут вернёт значение `1`).

`forloop.revcounter0` — количество итераций, оставшихся до конца цикла (на последнем проходе цикла этот атрибут вернёт значение `0`).

`forloop.first` — `True`, если это первая итерация.

`forloop.last` — `True`, если это последняя итерация.

Например, поставим HTML-тег `<hr>` (англ. horizontal rule, «горизонтальная линия») после последнего товара в списке:

***
```html
<!-- templates/catalog/product_list.html -->
...
<h1>{{ title }}</h1>
{% for product in products %}
  <p>{{ product }}</p>
  {% if forloop.last %}
    <hr>
  {% endif %}
{% empty %}
  Товаров пока не подвезли   
{% endfor %}
```
***

В циклах шаблонов можно применить необязательный одиночный тег `{% empty %}`; его ставят перед закрывающим тегом `{% endfor %}`. Если коллекция, переданная в цикл, пуста, будет выведено содержимое, размещённое между `{% empty %}` и `{% endfor %}`.

При рендеринге получится такой код:

***
```html
...
<h1>Список товаров ACME</h1>
<p>Iron carrot</p>
<p>Giant mousetrap</p>
<p>Dehydrated boulders</p>
<p>Invisible paint</p>
<hr> <!-- Этот тег будет вставлен только после последнего элемента. -->
...
```
***

Ссылка на [документацию по циклам в шаблонах](https://docs.djangoproject.com/en/3.2/ref/templates/builtins/#for) обязательно вам пригодится, сохраните её.